In [1]:
## Loading Config Files

import os 
import json
import sys

sys.path.append(os.path.dirname(os.getcwd()))
with open('../config/runner_config.json') as json_file:
    json_data = json.load(json_file)


In [2]:
## Data Aggreagator Run
from deployment.Data_Retreiver import Data_Retreiver
from deployment.Data_Aggregator import Aggregator

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

sql_table = json_data["sql_aggregate"]
sql_table_raw = json_data["sql_raw_energy"]
sql_table_state =  json_data["sql_raw_energy"]

# Create Threads for All the devices
aggre = Aggregator(devices,sql_table,sql_table_raw,sql_table_state,sql_addr,sql_port, sql_user, sql_pw, sql_db)

#Do step and get the final output that would be printed to the log
aggre.do_step()
print(aggre.getLatest())

---Removals of Data:Raw_Energy:0 Raw_State0 Aggregate0

Aggregator: ---Removals of Data:Raw_Energy:0 Raw_State0 Aggregate0



In [3]:
## Conctrol Enactor 
from deployment.Control_Enactor import Enactor


devices = json_data["devices"]

sftp_location = json_data["sftp_location"]
sftp_port = json_data["sftp_port"]
sftp_username = json_data["sftp_username"]
sftp_password = json_data["sftp_password"]
sftp_key_location = json_data["sftp_key_location"]
sftp_directory = json_data["sftp_directory"]

#Global for Unit Test
enact = Enactor(devices, sftp_location, sftp_port, sftp_username, sftp_password, sftp_key_location, sftp_directory)

# Run Examples
print("Retreive - Set Points")
print(enact.retreive_previous_plan_DC(list(devices.keys())[1]))
print(enact.retreive_previous_plan_AC(list(devices.keys())[9]))
print("Enact Set points")
print(enact.enact_light_plan(list(devices.keys())[1], 2100, 2100))
print(enact.enact_socket_plan(list(devices.keys())[9], 1000000))
print("Enact Same set points")
print(enact.enact_light_plan(list(devices.keys())[1], 2100, 2100))
print(enact.enact_socket_plan(list(devices.keys())[9], 1000000))
print("Enact Diffrent set points")
print(enact.enact_light_plan(list(devices.keys())[1], 4320, 4320))
print(enact.enact_socket_plan(list(devices.keys())[9], 1200000))


Retreive - Set Points
(4320, 4320)
1200000
Enact Set points
True
True
Enact Same set points
False
False
Enact Diffrent set points
True
True


In [4]:
#Data retreiver Run
import datetime
import pandas as pd 
from deployment.Data_Retreiver import Data_Retreiver
devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

print("--Retreive filled data--\n",data.get_unsent_logs(datetime.datetime.now()))
print("--Retreive filled data--\n",data.retreive_filled_aggre(datetime.datetime.now() - datetime.timedelta(days=20), 2))

dev_light = "Nursery_1A_CPE_No_2"
dev_socket = "Playground_AC_socket_No_1"
group = ["Nursery_1A_CPE_No_1", "Nursery_1A_CPE_No_2", "Nursery_1B_CPE_No_3", "Nursery_1B_CPE_No_4",
         "Nursery_1C_CPE_No_5", "Nursery_1C_CPE_No_6"]
hour = 6

pd.set_option('display.max_columns', 30)
dt_ts = datetime.datetime.now()
print("--Forecast--\n", data.retreive_latest_forecast(dt_ts).head(5))
print("--Raw Sys--\n", data.retreive_latest_raw_system_snapshot(dt_ts).head(5))
print("--Priority--\n", data.retreive_latest_priority(dt_ts).head(5))
print("--Group Energy--\n", data.get_total_energy_for_group(group, dt_ts))
print("--Aggregated Values --\n", data.retreive_aggregared_values(dt_ts))
print("--AVG for Hour Values --\n", data.retreive_average_vals_for_hour(dt_ts, hour,8))
print("--Average P Light--\n", data.retreive_average_P_lights(dev_light, dt_ts))
print("--AC Session--\n", data.retreive_AC_Session(dev_socket, dt_ts))
print("--AC Energy--\n", data.retreive_AC_Energy(dev_socket, dt_ts))
print("--Light Session--\n", data.retreive_Light_Session(dev_light, dt_ts))
print("--Light Energy--\n", data.retreive_Light_Energy(dev_light, dt_ts))
print("--Quota Latest Light--\n", data.get_latest_quota(dev_light, dt_ts))
print("--Quota Latest Socket --\n", data.get_latest_quota(dev_socket, dt_ts))



No Data Available for unsent logs 2020-09-08 08:17:02.347374  at:  2020-09-08 08:17:02.351122
--Retreive filled data--
 None
Value Error probably not data avilable for period:>  2020-08-19 08:15:00   2  Error: Length mismatch: Expected axis has 0 elements, new values have 37 elements
--Retreive filled data--
 None
--Forecast--
    system_load  generated_energy
0  2076.705918       2933.130727
--Raw Sys--
                   timestamp                        parameter       value
0 2020-09-08 08:00:00+00:00  VenusGX/Ac/Consumption/L1/Power  239.719970
1 2020-09-08 08:00:00+00:00         VenusGX/Dc/Battery/Power  665.806460
2 2020-09-08 08:00:00+00:00           VenusGX/Dc/Battery/Soc   98.287094
3 2020-09-08 08:00:00+00:00              VenusGX/Dc/Pv/Power  776.212460
--Priority--
    id                        timestamp  nursery1_lights  nursery1_sockets  \
0   2 2020-09-08 08:14:06.391048+00:00                1                 2   

   nursery2_lights  nursery2_sockets  playground_lights  

In [5]:
from deployment.Battery import Battery

bat = Battery(state_of_charge=94.0, battery_capacity=18.79, max_discharge=40.0, charge_efficiency=0.85)
print(bat)
print("---Storage Capacity---\n", bat.get_storage_capacity_left())
print("---Discharge Capacity---\n", bat.get_discharge_capacity_left())
print("---Discharge Battery---\n", bat.discharge_battery(1.9))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(0.4))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)


SoC: 94.0 Energy_Left: 10.1466
---Storage Capacity---
 1.3263529411764705
---Discharge Capacity---
 10.1466
---Discharge Battery---
 0.0
SoC: 83.88823842469398 Energy_Left: 8.246599999999999
---Charge Battery---
 0.0
SoC: 85.6977115486961 Energy_Left: 8.586599999999999
---Discharge Battery---
 0.0
SoC: 70.26397019691323 Energy_Left: 5.686599999999997
---Discharge Battery---
 0.0
SoC: 54.83022884513037 Energy_Left: 2.786599999999997
---Discharge Battery---
 0.11340000000000305
SoC: 40.0 Energy_Left: 0.0
---Discharge Battery---
 2.9
SoC: 40.0 Energy_Left: 0.0
---Discharge Battery---
 2.9
SoC: 40.0 Energy_Left: 0.0
---Charge Battery---
 0.0
SoC: 59.90420436402342 Energy_Left: 3.7400000000000007
---Charge Battery---
 0.0
SoC: 79.80840872804684 Energy_Left: 7.480000000000001
---Charge Battery---
 0.0
SoC: 99.71261309207026 Energy_Left: 11.22
---Charge Battery---
 4.336470588235296
SoC: 100.0 Energy_Left: 11.274
---Charge Battery---
 4.4
SoC: 100.0 Energy_Left: 11.274


In [6]:
# Controller Run

#Data retreiver Run
import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Control_Enactor import Enactor
from deployment.Controller import Controller

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

## Conctrol Enactor 
sftp_location = json_data["sftp_location"]
sftp_port = json_data["sftp_port"]
sftp_username = json_data["sftp_username"]
sftp_password = json_data["sftp_password"]
sftp_key_location = json_data["sftp_key_location"]
sftp_directory = json_data["sftp_directory"]

#Global for Unit Test
enact = Enactor(devices, sftp_location, sftp_port, sftp_username, sftp_password, sftp_key_location, sftp_directory)

#Controller Init
ts_step = datetime.datetime.now()  # - datetime.timedelta(hours=3)

allocation = json_data["allocation"]
if 'System_Data' in allocation:
    del allocation['System_Data']

control = Controller(data, enact, allocation, 4)
print(control.getLatest())

control.do_step(ts_step)
print(control.getLatest())

Controller : Initilised
Controller : Latest Data From: 2020-09-08 08:17:02.534533
-------Energy State--------
Generated energy: 2933.1307268589735
System Load: 2076.7059180000697
Battery SoC: 98.287094
Remaining Energy: 11509.80277585889
{1: 'nursery1_lights', 2: 'nursery1_sockets', 3: 'nursery2_lights', 4: 'nursery2_sockets', 5: 'playground_lights', 6: 'playground_sockets'}
------------------------------------------------
For Device: nursery1_lights with energy avialable: 11509.80277585889
Decisions: {'state': 'Unconstrained', 'energy_est_used_total': 146.46126186695096, 'constraining_factor': 1.0, 'device_const': {'Nursery_1A_CPE_No_1': (4320, 4320, False), 'Nursery_1A_CPE_No_2': (4320, 4320, False), 'Nursery_1B_CPE_No_3': (4320, 4320, False), 'Nursery_1B_CPE_No_4': (4320, 4320, False), 'Nursery_1C_CPE_No_5': (4320, 4320, False), 'Nursery_1C_CPE_No_6': (4320, 4320, False)}, 'timestamp': '2020-09-08 08:17:02.534533'}
Remaining: 11334.04926161855
---------------------------------------

In [7]:
## Forecaster V1 - Simple not ML

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Battery import Battery
from deployment.Forecaster import Forecaster

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)
forecast = Forecaster(Battery, data, devices)


curr_ts = datetime.datetime.now()
print("Running Forecasting -- ", curr_ts, " --")
forecast_period = 48
forecast.do_step(forecast_period, curr_ts)
print(forecast.getLatest())
#Latest Values
display(forecast.latest_forecast)

Running Forecasting --  2020-09-08 08:17:28.684553  --
Forecaster : 2020-09-08 08:17:34.230819 : [{"system_load":54.0304302621,"generated_energy":1065.3187255859,"consumed_energy":75.5845336914}]


,nursery_1a_cpe_no_1,nursery_1a_cpe_no_2,nursery_1b_cpe_no_3,nursery_1b_cpe_no_4,nursery_1c_cpe_no_5,nursery_1c_cpe_no_6,nursery_2a_cpe_no_7,nursery_2a_cpe_no_8,nursery_2b_cpe_no_10,nursery_2b_cpe_no_9,nursery_2c_cpe_no_11,nursery_2c_cpe_no_12,nursery_ac_socket_1a_no_1,nursery_ac_socket_1a_no_2,nursery_ac_socket_1b,...,playground_no_1,playground_no_2,playground_no_3,playground_no_4,playground_no_5,streetlight_no_1,streetlight_no_2,streetlight_no_3,battery_soc,charged_energy,consumed_energy,generated_energy,timestamp,system_load,victron_venusgx
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-09-08 07:00:00,0.528078,0.000000,0.313794,0.351357,0.016262,0.000000,0.014176,0.000000,0.021352,0.025981,0.0,0.000000,1.526820,0.318240,2.142241,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,97.891964,702.745094,73.467369,776.212463,2020-09-08 07:00:00,49.972033,0.0
2020-09-08 08:00:00,0.207418,0.000000,0.003997,0.000000,0.015764,0.000000,0.015994,0.000000,0.005430,0.085891,0.0,0.001000,1.257738,0.510468,2.295833,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,989.734192,75.584534,1065.318726,2020-09-08 08:00:00,54.030430,0.0
2020-09-08 09:00:00,0.162239,0.000000,0.000000,0.000000,0.015981,0.000000,0.016364,0.000000,0.026048,0.147785,0.0,0.000000,0.910119,0.219828,2.338492,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,812.771454,74.717499,887.488953,2020-09-08 09:00:00,56.796308,0.0
2020-09-08 10:00:00,0.030700,0.000000,0.000000,0.000000,0.016257,0.000000,0.016738,0.000000,0.018124,0.035014,0.0,0.000000,0.967857,0.241379,1.878571,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,614.202393,71.543457,685.745850,2020-09-08 10:00:00,56.350090,0.0
2020-09-08 11:00:00,0.000000,0.000000,0.000000,0.000000,0.016645,0.000000,0.016198,0.000000,0.114793,0.027637,0.0,0.000000,1.166872,0.278061,1.757266,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,352.319305,68.779236,421.098541,2020-09-08 11:00:00,50.130657,0.0
2020-09-08 12:00:00,0.000000,0.000000,0.000000,0.000000,0.016440,0.000000,0.016983,0.000981,0.068952,0.018524,0.0,0.000000,0.845170,0.185587,1.987685,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,220.756241,67.295212,288.051453,2020-09-08 12:00:00,50.035409,0.0
2020-09-08 13:00:00,0.000000,0.000000,0.000000,0.014201,0.014581,0.000000,0.014961,0.000000,0.001530,0.005344,0.0,0.000000,0.452176,0.325397,1.871191,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,100.000000,129.296000,96.751458,226.047458,2020-09-08 13:00:00,75.846745,0.0
2020-09-08 14:00:00,0.000000,0.000000,0.000000,0.000000,0.012597,0.000000,0.015089,0.000000,0.001980,0.019826,0.0,0.000000,0.205782,0.074074,1.442177,...,7.065510,6.719998,7.084459,6.699572,6.353567,7.334733,4.033067,5.773631,100.000000,49.857435,113.077569,162.935004,2020-09-08 14:00:00,47.553640,0.0
2020-09-08 15:00:00,2.671159,0.000000,2.733462,0.000000,2.902771,0.000000,3.124074,0.000000,7.793032,8.470631,0.0,2.795987,0.140394,0.000000,0.685345,...,12.523463,12.719450,12.692782,12.369589,12.464101,12.647728,12.892938,12.856978,100.000000,75.739208,62.905918,138.645126,2020-09-08 15:00:00,-77.983259,0.0


In [8]:
##Forecaster V2 - ML and PV Models
## Forecaster V1 - Simple not ML

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Battery import Battery
from deployment.Forecaster_v2 import Forecaster

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)
forecast = Forecaster(Battery, data, devices)


curr_ts = datetime.datetime.now()
print("Running Forecasting -- ", curr_ts, " --")
forecast_period = 48
forecast.do_step(forecast_period, curr_ts)
print(forecast.getLatest())
#Latest Values
display(forecast.full_df)

Running Forecasting --  2020-09-08 08:17:35.595390  --
Doing forecast for Dev : consumed_energy
Doing forecast for Dev : system_load
Doing forecast for Dev : streetlight_no_1
Doing forecast for Dev : streetlight_no_2
Doing forecast for Dev : streetlight_no_3
Doing forecast for Dev : playground_no_1
Doing forecast for Dev : playground_no_2
Doing forecast for Dev : playground_no_3
Doing forecast for Dev : playground_no_4
Doing forecast for Dev : playground_no_5
Doing forecast for Dev : playground_ac_socket_no_1
Doing forecast for Dev : playground_ac_socket_no_2
Doing forecast for Dev : nursery_ac_socket_1a_no_1
Doing forecast for Dev : nursery_ac_socket_1a_no_2
Doing forecast for Dev : nursery_ac_socket_1b
Doing forecast for Dev : nursery_ac_socket_1c
Doing forecast for Dev : nursery_1a_cpe_no_1
Doing forecast for Dev : nursery_1a_cpe_no_2
Doing forecast for Dev : nursery_1b_cpe_no_3
Doing forecast for Dev : nursery_1b_cpe_no_4
Doing forecast for Dev : nursery_1c_cpe_no_5
Doing forecast 

,consumed_energy,timestamp,system_load,streetlight_no_1,streetlight_no_2,streetlight_no_3,playground_no_1,playground_no_2,playground_no_3,playground_no_4,playground_no_5,playground_ac_socket_no_1,playground_ac_socket_no_2,nursery_ac_socket_1a_no_1,nursery_ac_socket_1a_no_2,...,nursery_1c_cpe_no_5,nursery_1c_cpe_no_6,nursery_ac_socket_2a_no_1,nursery_ac_socket_2a_no_2,nursery_ac_socket_2b,nursery_ac_socket_2c,nursery_2a_cpe_no_7,nursery_2a_cpe_no_8,nursery_2b_cpe_no_9,nursery_2b_cpe_no_10,nursery_2c_cpe_no_11,nursery_2c_cpe_no_12,generated_energy,charged_energy,battery_soc
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-09-08 09:00:00,64.443153,2020-09-08 09:00:00,47.204179,2.494875e-28,3.993786e-31,-1.001427e-30,-1.348222e-29,-1.533059e-29,2.376846e-29,3.599067e-29,-4.903950e-29,1.379895,2.855942,1.229406,0.156507,...,0.005000,-8.279614e-62,3.348912,4.095531,3.010172,0.645039,0.003100,0.0,0.031858,0.005815,0.0,7.333008e-35,1107.198235,1042.755082,4.20067
2020-09-08 10:00:00,69.170425,2020-09-08 10:00:00,50.379007,5.384154e-28,1.780678e-31,2.072967e-30,7.069454e-30,2.759108e-28,-1.014173e-29,2.137681e-29,6.094190e-30,1.194932,2.234279,0.894159,0.075234,...,0.005000,8.406571e-56,3.090975,3.442139,2.589294,-0.012662,0.003194,0.0,0.051723,0.012638,0.0,7.388202e-35,1098.673812,1029.503388,8.34796
2020-09-08 11:00:00,68.833923,2020-09-08 11:00:00,50.087785,-4.489551e-28,2.572729e-31,-1.486069e-28,1.998593e-31,-8.311140e-28,3.380990e-28,-2.684185e-29,1.340195e-30,0.849483,1.807235,0.948364,0.077023,...,0.005000,-8.535476e-50,2.737490,2.996912,2.347517,0.094399,0.004284,0.0,0.014598,0.009470,0.0,7.109444e-35,1004.984755,936.150832,12.1192
2020-09-08 12:00:00,68.868889,2020-09-08 12:00:00,44.136234,1.241382e-29,3.859445e-31,-2.265844e-29,-3.697388e-30,9.758353e-28,2.104642e-28,9.393598e-31,3.694432e-30,0.855553,1.906370,1.137882,0.083848,...,0.006000,8.666357e-44,2.892142,3.575656,3.070917,1.155673,0.003370,0.0,0.011476,0.041308,0.0,4.006798e-35,835.012210,766.143321,15.2055
2020-09-08 13:00:00,71.047078,2020-09-08 13:00:00,44.306175,-1.531714e-21,3.387550e-29,-3.100569e-27,1.564494e-30,-5.499603e-27,-1.183991e-21,-1.792715e-28,3.451713e-20,0.841849,0.937019,0.827849,0.057291,...,0.005000,-8.799245e-38,2.838774,3.648186,3.088671,1.068566,0.004451,0.0,0.008361,0.026155,0.0,8.475271e-35,604.408267,533.361189,17.3542
2020-09-08 14:00:00,142.227712,2020-09-08 14:00:00,77.204505,-5.506018e-11,-1.603244e-29,-1.040130e-27,-5.503573e-31,1.980612e-16,-9.129680e-17,9.870794e-29,8.893815e-11,2.943420,1.866483,0.449809,0.099143,...,0.004500,8.934173e-32,2.468618,2.624306,2.129779,0.200374,0.003779,0.0,0.004000,0.004010,0.0,1.288699e-34,338.010817,195.783105,18.1429
2020-09-08 15:00:00,192.772183,2020-09-08 15:00:00,103.438159,2.449932e+00,1.375130e+00,1.919864e+00,2.233157e+00,2.230421e+00,2.254624e+00,2.111117e+00,2.076833e+00,0.970277,4.425392,0.217042,0.053728,...,0.003750,-9.086228e-26,2.513747,2.290553,1.641528,0.493892,0.003102,0.0,0.007395,0.003870,0.0,1.221049e-34,84.858867,-107.913316,40
2020-09-08 16:00:00,12.601718,2020-09-08 16:00:00,4.091435,4.216002e+00,4.298002e+00,4.286005e+00,4.173996e+00,4.239993e+00,4.230998e+00,4.123006e+00,4.155001e+00,1.433778,2.514227,0.154259,0.003943,...,0.968003,9.838154e-19,1.602303,2.371357,1.660653,-0.003536,1.039671,0.0,2.826043,2.600735,0.0,9.320019e-01,0.004895,-12.596823,40
2020-09-08 17:00:00,91.563522,2020-09-08 17:00:00,75.135033,4.148002e+00,4.169002e+00,4.268005e+00,4.055997e+00,4.270993e+00,4.247998e+00,4.076006e+00,4.085001e+00,0.615648,2.425317,0.197318,0.003084,...,1.612005,1.099999e-02,1.325476,1.768737,1.354414,-0.002859,1.597737,0.0,4.784947,4.617606,0.0,1.594003e+00,0.000000,-91.563522,40


In [9]:
##Remote Logger Example

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Remote_Logger import RemoteLogger

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

mqtt_address = json_data["mqtt_address"]
mqtt_port = json_data["mqtt_port"]
mqtt_username = json_data["mqtt_username"]
mqtt_password = json_data["mqtt_password"]

data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

rl = RemoteLogger(data, mqtt_address, mqtt_port, mqtt_username, mqtt_password)

rl.do_step()
print(rl.getLatest())

Remote Logger: 2020-09-08 08:20:09.453505 : Publishing message with Nulls:['quota2_nursery_ac_socket_1a_no_1', 'quota2_nursery_ac_socket_1a_no_2', 'quota2_nursery_ac_socket_1b', 'quota2_nursery_ac_socket_1c', 'quota2_nursery_ac_socket_2a_no_1', 'quota2_nursery_ac_socket_2a_no_2', 'quota2_nursery_ac_socket_2b', 'quota2_nursery_ac_socket_2c', 'quota2_playground_ac_socket_no_1', 'quota2_playground_ac_socket_no_2']  with return:(0, 1)
